# TP1

In [11]:
import numpy as np
import pandas as pd
import pydotplus
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Image
from IPython.display import display
from sklearn import preprocessing
from sklearn.externals.six import StringIO  
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix


## Lectura del dataset

Se reemplaza el atributo "Data of birth" por "Age", encodeamos "Social Class". También encodeamos "Presence of Children" para que tome 2 valores: 0 si no hay hijos, 1 en caso contrario

In [6]:
def readCsv():
    df = pd.read_csv("FullData2.csv")
    
    #Agregamos atributo edad
    df['Date of birth'] = pd.to_datetime('today').year - pd.to_datetime(df['Date of birth'],format='%Y-%m-%d').dt.year
    df = df.rename(columns={'Date of birth': 'Age' })
    
    #Encode social class
    socialClassEncoder = preprocessing.LabelEncoder()
    socialClassEncoder.fit(["AB", "C1", "C2", "D ", "E "])
    
    df['Social Class'] = socialClassEncoder.transform(df['Social Class'])
    
    #Agrupamos en 2 clases "Presence of Children"
    df['Presence of Children'] = df['Presence of Children'].apply(lambda x: 1 if x > 1 else 0)
    
    return df

df = readCsv()

## 1. Partición de datos 

Particionamos el conjunto entre datos de entramiento y de test. Los de validación se van a ir generando en cada uno de los k-folds. 

In [7]:
def partitionate(df):
    #Seteamos Presence of Children como target
    targetColumn = "Presence of Children"
    
    #El resto de las columnas las vamos a usar como feature para clasificar
    features = list(df.columns)
    features.remove(targetColumn)
    
    #Eliminamos otras columnas que están muy relacionadas con lo que queremos predecir (deberíamos sacarlas del csv)
    features.remove("Demographic cell 1")
    features.remove("Mosaic Classification")
    features.remove("Life stage")
    features.remove("No of People")
    features.remove("Age")
    features.remove("Terminal age of education")

    return train_test_split(df[features], df[targetColumn], train_size=0.8, test_size=0.2, random_state=42)

X, test_X, y, test_y = partitionate(df)

# Comparación de algoritmos
Comparar Naive Bayes y árboles de decisión. Para hacerlo usar 5-fold crossvalidation para la exploración de la mejor solución en cada caso. En árboles de
decisión determinar qué tamaño de árbol conviene y si conviene utilizar Gini o
Information Gain (todo esto con el conjunto de desarrollo y utilizando grid search).
Utilizar ROC AUC como métrica.

In [42]:
# se instancia el modelo NB 
model = GaussianNB()
model.fit(X,y)

# en train
scores = cross_val_score(model, X, y, cv=5, scoring='roc_auc')
print('scoring ROC train:   ',round(100*scores.mean(),2),'%')
predict_test=model.predict(test_X)
print('scoring ROC test :   ',round(roc_auc_score(test_y, predict_test)*100,2),'%')

scoring ROC train:    79.44 %
scoring ROC test :    68.68 %


In [43]:
print('Repote de métricas:')
print (classification_report(test_y, predict_test))

Repote de métricas:
              precision    recall  f1-score   support

           0       0.84      0.54      0.66      1614
           1       0.52      0.83      0.64       966

   micro avg       0.65      0.65      0.65      2580
   macro avg       0.68      0.69      0.65      2580
weighted avg       0.72      0.65      0.65      2580



In [44]:
# se optimiza el modelo de AD
profundidad = list(range(1, 41))
criterio = ["gini","entropy"]
split=['best','random']

#instanciamos el modelo
tree = DecisionTreeClassifier(max_depth = 3, criterion = 'gini',splitter='best')
        
param_grid = dict(max_depth=profundidad, criterion=criterio,splitter=split)
grid = GridSearchCV(tree, param_grid, cv=5, scoring='roc_auc')
grid.fit(X, y)

print (grid.best_estimator_)
print()
print('mejor estimación:   ',round(grid.best_score_*100,2),'%')

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=6,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

mejor estimación:    86.23 %


In [45]:
# estimamos con los parámetros optimos
predict_test=grid.predict(test_X)
print('scoring ROC test :   ',round(roc_auc_score(test_y, predict_test)*100,2),'%')
print()
print('Reporte de metricas:')
print (classification_report(test_y, predict_test))
print()
print('Matriz de confusión:')
print(confusion_matrix(test_y, predict_test))

scoring ROC test :    78.18 %

Reporte de metricas:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85      1614
           1       0.78      0.68      0.73       966

   micro avg       0.81      0.81      0.81      2580
   macro avg       0.80      0.78      0.79      2580
weighted avg       0.81      0.81      0.80      2580


Matriz de confusión:
[[1431  183]
 [ 312  654]]
